In [16]:
!wget http://vision.ime.usp.br/~arturao/cenipa/Cenipa.zip
!apt install unzip
!unzip Cenipa.zip

--2021-12-20 18:20:27--  http://vision.ime.usp.br/~arturao/cenipa/Cenipa.zip
Resolving vision.ime.usp.br (vision.ime.usp.br)... 143.107.44.18
Connecting to vision.ime.usp.br (vision.ime.usp.br)|143.107.44.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43666843 (42M) [application/zip]
Saving to: ‘Cenipa.zip.1’

Cenipa.zip.1        100%[===================>]  41.64M  16.8MB/s    in 2.5s    

2021-12-20 18:20:31 (16.8 MB/s) - ‘Cenipa.zip.1’ saved [43666843/43666843]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Archive:  Cenipa.zip
replace Cenipa/2010_2019_aerodromos.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
!pip install neo4j --quiet

In [18]:
import geopy.distance
import pandas as pd
import plotly.express as px

# Data Loading

In [19]:
FOLDER_PREFIX = "Cenipa" #folder containing csvs and such
airplanes = pd.read_csv(f"./{FOLDER_PREFIX}/aeronave.csv", sep=';')
factor = pd.read_csv(f"./{FOLDER_PREFIX}/fator_contribuinte.csv", sep=';')
occurence = pd.read_csv(f"./{FOLDER_PREFIX}/ocorrencia.csv", sep=';')
occurence_type = pd.read_csv(f"./{FOLDER_PREFIX}/ocorrencia_tipo.csv", sep=';')
recomendation = pd.read_csv(f"./{FOLDER_PREFIX}/recomendacao.csv", sep=';')

## Brief Visualization of the Data

In [20]:
airplanes.head()

,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,39115,PTNQX,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-711A,P28R,PISTÃO,MONOMOTOR,1202,1202,4.0,1979.0,BRASIL,BRASIL,AVIÃO,PARTICULAR,BRIGADEIRO ARARIPE MACEDO,CORRENTINA,DECOLAGEM,PRIVADA,SUBSTANCIAL,0
1,39155,PTLVI,***,AVIÃO,BEECH AIRCRAFT,C90,BE9L,TURBOÉLICE,BIMOTOR,4377,4377,8.0,1979.0,BRASIL,BRASIL,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,PRIVADA,NENHUM,0
2,39156,PPPTO,***,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500,22500,73.0,2008.0,BRASIL,BRASIL,AVIÃO,REGULAR,AFONSO PENA,ADALBERTO MENDES DA SILVA,ARREMETIDA NO AR,REGULAR,LEVE,0
3,39158,PRLGJ,REGULAR,AVIÃO,BOEING COMPANY,757-225,B752,JATO,BIMOTOR,108862,108862,5.0,1984.0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA,REGULAR,NENHUM,0
4,39176,PRMAA,REGULAR,AVIÃO,AIRBUS INDUSTRIE,A320-232,A320,JATO,BIMOTOR,77000,77000,184.0,2001.0,BRASIL,BRASIL,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA,REGULAR,NENHUM,0


In [21]:
factor.head()

,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,39115,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,39115,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
2,39115,PERCEPÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
3,39115,PLANEJAMENTO DE VOO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
4,39115,POUCA EXPERIÊNCIA DO PILOTO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL


In [22]:
occurence.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,BRASIL,****,03/01/2010,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,03/01/2010,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,03/01/2010,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,BRASIL,****,04/01/2010,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,05/01/2010,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


In [23]:
occurence_type.head()

,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,39115,PANE SECA,COMBUSTÍVEL | PANE SECA,FUEL
1,39155,VAZAMENTO DE COMBUSTÍVEL,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,39156,FOGO EM VOO,FOGO/FUMAÇA (SEM IMPACTO) | FOGO EM VOO,F-NI
3,39158,SUPERAQUECIMENTO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | SUPERAQU...,SCF-PP
4,39176,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP


In [24]:
recomendation.head()

,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,39115,095/2011,2011-07-21,2011-08-01,2015-12-21,Revisar os requisitos de experiência recente p...,NÃO CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,39115,096/2011,2011-07-21,2011-08-01,2011-09-16,Divulgar o presente relatório final aos operad...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,39156,063/2011,2011-06-30,2011-08-01,2011-09-12,Divulgar este Relatório aos operadores de aero...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,39156,064/2011,2011-06-30,2011-08-01,2014-04-16,Acompanhar junto à autoridade primária de cert...,CUMPRIDA DE FORMA ALTERNATIVA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,39235,205/2011,2011-08-29,2011-09-29,2012-11-06,Realizar uma vistoria de segurança de voo na A...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


# Data Analysis

On this data analysis there are some questions which I'd like to answer



1.   When an accident happen, in which phase it is most likely to happen?
2.   Is there a correlation between the distance in flights and the quantity of accidents?
3.   Which manufacter has had the most ammount of accidents in their planes? Which is the faultiest/unluckiest airplane model?
4.   Is there a sasonality/a season in which accidents tend to happen?
5.   Are the number of fatalities directly linked to the damage caused to the aircraft?



## When an accident happen, in which phase it is most likely to happen?

In [25]:
airplanes.groupby("aeronave_fase_operacao").count().codigo_ocorrencia2.sort_values(ascending=False)

aeronave_fase_operacao
POUSO                              1034
DECOLAGEM                          1011
CRUZEIRO                            926
CORRIDA APÓS POUSO                  656
TÁXI                                457
SUBIDA                              447
APROXIMAÇÃO FINAL                   317
MANOBRA                             247
DESCIDA                             175
ESPECIALIZADA                       162
CIRCUITO DE TRÁFEGO                 147
INDETERMINADA                       112
OUTRA FASE                           83
ESTACIONAMENTO                       82
VOO A BAIXA ALTURA                   62
ARREMETIDA NO SOLO                   47
OPERAÇÃO DE SOLO                     33
ARREMETIDA NO AR                     31
PARTIDA DO MOTOR                     29
***                                  26
PAIRADO                              19
PROCEDIMENTO DE APROXIMAÇÃO IFR      17
RETA FINAL                           17
DECOLAGEM VERTICAL                   13
SAÍDA IFR        

After a quick glance at the data, you could be induced to believe there is not a big difference between the three main phases of the flight in terms of crash percentage. But, if you take in account sub phases of Departure and Arrival, clearly the Cruise phase is the safest one.

We will manually create a mapperc below that maps each category to these 3 phases.

In [26]:
state_mapper = {
    "POUSO": "arrival",
    "DECOLAGEM": "departure",
    "CRUZEIRO": "cruise",
    "CORRIDA APÓS POUSO": "arrival",
    "TÁXI": "uncertain",
    "SUBIDA": "departure",
    "APROXIMAÇÃO FINAL": "arrival",
    "MANOBRA": "arrival",
    "DESCIDA": "arrival",
    "ESPECIALIZADA": "uncertain",
    "CIRCUITO DE TRÁFEGO": "uncertain",
    "INDETERMINADA": "uncertain",
    "OUTRA FASE": "uncertain",
    "ESTACIONAMENTO": "arrival",
    "VOO A BAIXA ALTURA": "cruise",
    "ARREMETIDA NO SOLO": "departure",
    "OPERAÇÃO DE SOLO": "departure",
    "ARREMETIDA NO AR": "arrival",
    "PARTIDA DO MOTOR": "departure",
    "***": "uncertain",
    "PAIRADO": "uncertain",
    "PROCEDIMENTO DE APROXIMAÇÃO IFR": "arrival",
    "RETA FINAL": "departure",
    "DECOLAGEM VERTICAL": "departure",
    "SAÍDA IFR": "departure",
    "CHEQUE DE MOTOR OU ROTOR": "departure",
    "PUSHBACK": "uncertain",
    "ESPERA": "uncertain",
    "DESCIDA DE EMERGÊNCIA": "arrival",
    "DESCIDA DESCONTROLADA": "arrival",
    "EMPREGO MILITAR": "uncertain",
    "MUDANÇA DE NÍVEL OU ALTITUDE": "cruise"
}

airplanes['phase'] = airplanes['aeronave_fase_operacao'].apply(lambda x: state_mapper[x])

In [27]:
airplanes.groupby("phase").count().codigo_ocorrencia2.sort_values(ascending=False)

phase
arrival      2565
departure    1614
uncertain    1020
cruise        989
Name: codigo_ocorrencia2, dtype: int64

Now we can see a little bit clearer how well distributed these phases are. If we consider the uncertain phase (which was created due to the author of this study not being sure where to put some phases), it is clear that the arrival and departure phases are more accident prone than the cruise.

Might even risk saying that the arrival phase is the most dangerous part of the flight

## Is there a correlation between the distance in flights and the quantity of accidents?

In [28]:
airplanes.query("aeronave_voo_origem != 'FORA DE AERODROMO' and aeronave_voo_destino != 'FORA DE AERODROMO'")

,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,phase
0,39115,PTNQX,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-711A,P28R,PISTÃO,MONOMOTOR,1202,1202,4.0,1979.0,BRASIL,BRASIL,AVIÃO,PARTICULAR,BRIGADEIRO ARARIPE MACEDO,CORRENTINA,DECOLAGEM,PRIVADA,SUBSTANCIAL,0,departure
2,39156,PPPTO,***,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500,22500,73.0,2008.0,BRASIL,BRASIL,AVIÃO,REGULAR,AFONSO PENA,ADALBERTO MENDES DA SILVA,ARREMETIDA NO AR,REGULAR,LEVE,0,arrival
7,39275,PTYRE,***,HELICÓPTERO,EUROCOPTER FRANCE,AS 350 BA,***,TURBOEIXO,MONOMOTOR,2100,2100,6.0,1994.0,BRASIL,BRASIL,HELICÓPTERO,PARTICULAR,CAMPO DE MARTE,CAMPO DE MARTE,CRUZEIRO,PRIVADA,DESTRUÍDA,1,cruise
10,39316,PPGMH,***,AVIÃO,AERO BOERO,AB-115,AB11,PISTÃO,MONOMOTOR,770,770,2.0,1992.0,BRASIL,BRASIL,AVIÃO,INSTRUÇÃO,BLUMENAU,RANCHO SUMIDOR,POUSO,INSTRUÇÃO,SUBSTANCIAL,0,arrival
12,39318,PTTAF,***,AVIÃO,EMBRAER,EMB-110,E110,TURBOÉLICE,BIMOTOR,5600,5600,1.0,1976.0,BRASIL,BRASIL,AVIÃO,TÁXI AÉREO,VAL DE CANS / JÚLIO CEZAR RIBEIRO,WILMA REBELO,DESCIDA,TÁXI AÉREO,SUBSTANCIAL,2,arrival
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6183,80249,PRAJY,***,AVIÃO,BOEING COMPANY,737-48E,737,JATO,BIMOTOR,65090,65090,192.0,NaN,BRASIL,BRASIL,AVIÃO,REGULAR,EDUARDO GOMES,VIRACOPOS,SUBIDA,REGULAR,NENHUM,0,departure
6184,80251,PRAJY,***,AVIÃO,BOEING COMPANY,737-48E,737,JATO,BIMOTOR,65090,65090,192.0,NaN,BRASIL,BRASIL,AVIÃO,REGULAR,VAL DE CANS / JÚLIO CEZAR RIBEIRO,PINTO MARTINS,SUBIDA,REGULAR,NENHUM,0,departure
6185,80257,PRLBR,***,HELICÓPTERO,ROBINSON HELICOPTER,R44 II,R44,PISTÃO,MONOMOTOR,1134,1134,4.0,2010.0,BRASIL,BRASIL,HELICÓPTERO,PARTICULAR,***,***,***,PRIVADA,DESTRUÍDA,0,uncertain
6186,80258,PTRXE,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-721D,P32R,PISTÃO,MONOMOTOR,1633,1633,7.0,1985.0,BRASIL,BRASIL,AVIÃO,PARTICULAR,TANGARÁ DA SERRA,PILOTO OSVALDO MARQUES DIAS,DECOLAGEM,PRIVADA,SUBSTANCIAL,0,departure


Utilizing a previously generated query to an API that can geocode the airport names presented in the airplanes csv, we may be able to see if our question holds true

In [29]:
# import pre-generated mapper from airport to lat long
with open("./airport_geoloc.pkl", "rb") as f:
  mapped_airports = pickle.load(f)

NameError: ignored

In [ ]:
airplanes_geoloc = pd.merge(left=airplanes, right=pd.DataFrame(mapped_airports).T.reset_index(), left_on="aeronave_voo_origem", right_on="index", suffixes=(None, "_departure"))
airplanes_geoloc = pd.merge(left=airplanes_geoloc, right=pd.DataFrame(mapped_airports).T.reset_index(), left_on="aeronave_voo_destino", right_on="index", suffixes=(None, "_arrival")).drop(["index", "index_arrival"], axis=1).dropna()


In [ ]:
airplanes_geoloc['coords'] = list(
    zip(
        list(zip(airplanes_geoloc.latitude, airplanes_geoloc.longitude)), 
        list(zip(airplanes_geoloc.latitude_arrival, airplanes_geoloc.longitude_arrival))
    )
)

airplanes_geoloc['distance'] = airplanes_geoloc.coords.apply(
    lambda x: geopy.distance.vincenty(x[0], x[1]).km
)


In [ ]:
px.histogram(airplanes_geoloc, x='distance', nbins=50)

We can note that, for the majority of flights that have both the departure and arrival aerodome registered, the distance is 0, meaning that the arrival and departure airports are the same. Resulting in a 0 distance flight

In [ ]:
px.histogram(airplanes_geoloc.query("distance > 100"), x='distance', nbins=50)

And even if we filter it at 100 km, most flights that have accidents registered by Cenipa seen to happen for close flights. This could possibly indicate that the correlation exists and is inverse to the distance.

This makes sense if we take into account previous knowledge of flight accidents, seemingly most happen during departure and arrival procedures, and since close flights tend to happen more often they are more succeptible to failure

## Which manufacter has had the most ammount of accidents in their planes? Which is the faultiest/unluckiest airplane model?

In [ ]:
airplanes.groupby('aeronave_fabricante').count()['codigo_ocorrencia2'].sort_values(ascending=False)

Lets filter by the top 3 manufacturers.

In [ ]:
manufacturers = list(airplanes.groupby('aeronave_fabricante').count()['codigo_ocorrencia2'].sort_values(ascending=False).head(3).index)
filtered_airplanes = airplanes.query("aeronave_fabricante in @manufacturers")

In [ ]:
filtered_airplanes.head(5)

After a quick-search, I was able to make two interesting discoveries

Indústria Aeronáutica Neiva is a subsidiary of Embraer and the most employed agricultural aircraft in Brazil

and Cessna Aircraft Company was an American general aviation aircraft manufacturing corporation headquartered in Wichita, Kansas. The company produced small, piston-powered aircraft, as well as business jets. Since 2014 it has been declared defunct after joining three other distinct brands as part of Textron Aviation

Due to this discovery, Embraer is the manufacturer with the most occurences during this time frame.

In [ ]:
grouped = airplanes.groupby('aeronave_fabricante').count()['codigo_ocorrencia2'].sort_values(ascending=False).reset_index()
grouped.loc[grouped['codigo_ocorrencia2'] < 50, 'aeronave_fabricante'] = 'Other'
px.pie(grouped, values='codigo_ocorrencia2', names='aeronave_fabricante', title='Airplanes in Occurences')

In [ ]:
filtered_airplanes.groupby('aeronave_modelo').count()['codigo_ocorrencia2'].sort_values(ascending=False).head()

In [ ]:
filtered_airplanes.query("aeronave_modelo == 'ERJ 190-200 IGW'").head(1)

The model ERJ 190-200 IGW from Embraer is the number one in quantity of occurences, having appeared 183 times. It is a commercial airplane with 125 seats and has around [1500 models built](https://daflwcl3bnxyt.cloudfront.net/m/558ea4784621a83/original/Deliveries-Backlog-1Q21-INS-VRF-ENG-21-Final-rev1.pdf)


## Is there a sasonality/a season in which accidents tend to happen?

While we study the seasonality of these events, we will be dealing with MONTH/YEAR pairs, since accidents don't happen daily.

In [ ]:
occurence_copy = occurence.copy()
occurence_copy['ocorrencia_dia'] = pd.to_datetime(occurence_copy['ocorrencia_dia'], format='%d/%m/%Y')
occurence_copy['occorencia_hora'] = pd.to_datetime(occurence_copy['ocorrencia_hora'], format='%H:%M:%S')

In [ ]:
occurence_copy['month_year'] = occurence_copy['ocorrencia_dia'].dt.to_period('M')
occurence_copy['hour'] = occurence_copy['occorencia_hora'].apply(lambda x: x.hour)

In [ ]:
grouped = occurence_copy.groupby('month_year').count()['codigo_ocorrencia'].reset_index()
grouped['month_year'] = grouped['month_year'].astype(str)
px.line(grouped, x="month_year", y="codigo_ocorrencia", title='Number of Accidents by Time Frame')

The graph above seems to variate too much from a one month to the other, which may cause some difficulty to find/understand any feasible seasonality that may occur, so, we will apply a rolling average of 3 months to soften the curve

In [ ]:
grouped = occurence_copy.groupby('month_year').count()['codigo_ocorrencia'].rolling(3).mean().dropna().reset_index()
grouped['month_year'] = grouped['month_year'].astype(str)
px.line(grouped, x="month_year", y="codigo_ocorrencia", title='Number of Accidents by Time Frame')

There isn't really any kind of seasonality when we look into the accidents in an date aspect, we could try looking into it by an hour analysis

In [ ]:
grouped = occurence_copy.groupby('hour').count()['codigo_ocorrencia'].reset_index()
grouped['hour'] = grouped['hour'].astype(str)
px.line(grouped, x="hour", y="codigo_ocorrencia", title='Number of Accidents by Time Frame')

Most accidents seem to happen at the commercial time. Probably due to most flights leaving/arriving at their destinations in this time frame.

## Are the number of fatalities directly linked to the damage caused to the aircraft? Which is the most common failure to happen to the destroyed aircrafts?

In [ ]:
airplanes.groupby('aeronave_nivel_dano').sum()[['aeronave_fatalidades_total']]

Basically, yes, destroyed aircrafts usually leads to more fatalities.

But, which is the most common failure among those?

In [ ]:
grouped = (
    pd.merge(
        left=airplanes.query("aeronave_nivel_dano == 'DESTRUÍDA'"),
        right=occurence_type,
        left_on='codigo_ocorrencia2',
        right_on='codigo_ocorrencia1'
    ).groupby('ocorrencia_tipo_categoria')
      .sum()[['aeronave_fatalidades_total']]
      .sort_values(by='aeronave_fatalidades_total', ascending=False)
      .head(5)
)

In [ ]:
grouped

Como esperado, colisões normalmente acabam causado maior destruição das aeronaves
